In [ ]:
#libraries to import
import socket
import logging
import boto3
from botocore.exceptions import ClientError

ec2_client = boto3.client('ec2', region_name='us-east-1')
ec2_resrc=boto3.resource('ec2',region_name='us-east-1') 

#ec2 functionalities

#definitions of constants
MAX_APP_EC2=19
MIN_APP_EC2=0
KEY_PAIR=0
SEC_GROUP_ID=0
APP_TIER_PREFIX='app_tier_ec2'

#launch ec2 instance
#image id is the AMI given in the project description on Canvas

def create_instance(key_pair,security_group_id, img_id='ami-0bb1040fdb5a076bc',min=1,max=1):
    instance_list=ec2_resrc.create_instances(
        ImageID=img_id,
        MinCount=min,
        MaxCount=max,
        InstanceType='t2.micro',
        KeyName=key_pair,
        SecurityGroupIds=[security_group_id],
        #UserData=USERDATA, --> needed to start web tier automatically (write bash script --> automates the process of web tier starting .sh file )
        #IamInstanceProfile=INSTANCE_PROFILE --> google
    )

    #creating instance tags for each instance created by storing values as {'app_tier_ec2',instance number} until we arrive at 19 instances
    for i in range(max):
        j=i+1
        inst_id=instance_list[i]
        print(inst_id)
        ec2_resrc.create_tags(Resources=[inst_id],
        Tags=[{
            'Key':'Name',
            'Value':'{}{}'.format(APP_TIER_PREFIX,j)
        },
        ])






    



    
